# Day 4: Giant Squid

In [1]:
from pathlib import Path
import re
from dataclasses import dataclass

from aoc2021.util import read_as_list

## Puzzle input data

In [2]:
parse_line = lambda x: re.split('[,\s]+', x.strip())

# Test data.
tdata = list(map(parse_line, [
            '7,4,9,5,11,17,23,2,0,14,21,24,10,16,13,6,15,25,12,22,18,20,8,19,3,26,1',
            '',
            '22 13 17 11  0',
            ' 8  2 23  4 24',
            '21  9 14 16  7',
            ' 6 10  3 18  5',
            ' 1 12 20 15 19',
            '',
            ' 3 15  0  2 22',
            ' 9 18 13 17  5',
            '19  8  7 25 23',
            '20 11 10 24  4',
            '14 21 16 12  6',
            '',
            '14 21 17 24  4',
            '10 16 15  9 19',
            '18  8 23 26 20',
            '22 11 13  6  5',
            ' 2  0 12  3  7',
        ]
    )
)

# Input data.
data = read_as_list(Path('./day04.txt'), func=parse_line)
data[0][:12], data[1:8]

(['15', '61', '32', '33', '87', '17', '56', '73', '27', '83', '0', '18'],
 [[''],
  ['26', '68', '3', '95', '59'],
  ['40', '88', '50', '22', '48'],
  ['75', '67', '8', '64', '6'],
  ['29', '2', '73', '78', '5'],
  ['49', '25', '80', '89', '96'],
  ['']])

## Puzzle answers
### Part 1

In [3]:
@dataclass
class Board:
    grid: list[list]

    @property
    def vals(self):
        return (x for xs in self.grid for x in xs)

    @property
    def rows(self):
        return (row for row in self.grid)

    @property
    def cols(self):
        return map(list, zip(*self.grid))
    
    @property
    def size(self):
        return len(list(self.rows))


def sublists(xs: list, n: int) -> list[list]:
    return [xs[i:i+n] for i in range(0, len(xs), n)]


def parse_data(data) -> tuple[list[int], list[Board]]:
    nums = list(map(int, data[0]))
    boards_rows = [list(map(int, row)) for row in data[1:] if row != ['']]
    boards = list(map(Board, sublists(boards_rows, 5)))
    return nums, boards


def update(mb: Board, b: Board, n: int) -> Board:
    if n not in b.vals:
        return mb
    marked_grid = sublists([marked or (n == num) for marked, num in zip(mb.vals, b.vals)], b.size)
    return Board(marked_grid)


def bingo(b: Board) -> bool:
    return any(map(all, b.rows)) or any(map(all, b.cols))


def unmarked_nums(b: Board, mb: Board) -> list[int]:
    return [num for num, is_marked in zip(b.vals, mb.vals) if not is_marked]


def board_score(b: Board, mb: Board, n: int) -> int:
    return n * sum(unmarked_nums(b, mb))


def solve(data: list) -> int:
    nums, boards = parse_data(data)
    marked_boards = [Board([[False]*5]*5) for _ in range(len(boards))]
    for n in nums:
        marked_boards = [update(mb, b, n) for mb, b in zip(marked_boards, boards)]
        if bingos := [ii for ii, mb in enumerate(marked_boards) if bingo(mb)]:
            idx = bingos[0]
            winner_b = boards[idx]
            winner_mb = marked_boards[idx]
            return board_score(winner_b, winner_mb, n)


assert list(Board([[11,12],[21,22]]).vals) == [11,12,21,22]
assert list(Board([[11,12],[21,22]]).rows) == [[11,12],[21,22]]
assert list(Board([[11,12],[21,22]]).cols) == [[11,21],[12,22]]
assert sublists([0,1,2,3,4,5], 2) == [[0,1],[2,3],[4,5]]
assert parse_data(tdata)[0] == [7,4,9,5,11,17,23,2,0,14,21,24,10,16,13,6,15,25,12,22,18,20,8,19,3,26,1]
assert parse_data(tdata)[1] == [
    Board([[22,13,17,11,0],[8,2,23,4,24],[21,9,14,16,7],[6,10,3,18,5],[1,12,20,15,19]]),
    Board([[3,15,0,2,22],[9,18,13,17,5],[19,8,7,25,23],[20,11,10,24,4],[14,21,16,12,6]]),
    Board([[14,21,17,24,4],[10,16,15,9,19],[18,8,23,26,20],[22,11,13,6,5],[2,0,12,3,7]]),
]
assert bingo(Board([[True,False],[True,True]])) == True
assert bingo(Board([[True,False],[False,True]])) == False
assert unmarked_nums(Board([[11,12],[21,22]]), Board([[True,False],[True,False]])) == [12,22]
assert update(Board([[True,False],[False,False]]), Board([[11,12],[21,22]]), 21) == Board([[True,False],[True,False]])
assert solve(tdata) == 4512

In [4]:
score = solve(data)
print(f'The final score of the winning board: {score}')

The final score of the winning board: 58412


### Part 2

In [5]:
def solve(data: list) -> int:
    nums, boards = parse_data(data)
    marked_boards = [Board([[False]*5]*5) for _ in range(len(boards))]
    for n in nums:
        marked_boards = [update(mb, b, n) for mb, b in zip(marked_boards, boards)]
        if bingos := [ii for ii, mb in enumerate(marked_boards) if bingo(mb)]:
            if len(boards) == 1:
                return board_score(boards[0], marked_boards[0], n)
            boards = [b for ii, b in enumerate(boards) if ii not in bingos]
            marked_boards = [mb for ii, mb in enumerate(marked_boards) if ii not in bingos]


assert solve(tdata) == 1924

In [6]:
score = solve(data)
print(f'The final score of the last board to win: {score}')

The final score of the last board to win: 10030
